In [11]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import timedelta

In [12]:
transactions = [
    {'date': '2020-07-02', 'action': 'buy', 'ticker': 'SOXX', 'share': 2, 'price': 300.0}, #buy
    {'date': '2021-08-20', 'action': 'sell', 'ticker': 'SOXX', 'share': 2, 'price': 320.0}, #sell
    {'date': '2020-07-24', 'action': 'dividendCash', 'ticker': 'SOXQ', 'amount': 3 }, #cash dividend
    {'date': '2020-08-20', 'action': 'adjustShare', 'ticker': 'SOXX', 'share': 1}, #share dividend/ merge/ split
]

In [19]:
def transactionRecordsToTickersBalance(transactions):

    def transactionRecordToAccountMovement(transaction):

        date = pd.to_datetime(transaction['date'])
        action = transaction['action']
        ticker = transaction['ticker']
        share = transaction.get('share', 0)
        price = transaction.get('price', 0)
        amount = transaction.get('amount', 0)
        fee = transaction.get('fee', 0)

        shareChange = 0
        cashChange = 0

        if action == 'buy':
            shareChange = share
            cashChange = -price * share - fee

        if action == 'sell':
            shareChange = -share
            cashChange = price * share + fee

        if action == 'dividendCash':
            cashChange = amount

        if action == 'adjustShare':
            shareChange = share


        return {
          'date': date, 
          'ticker': ticker,
          'shareChange': shareChange, 
          'cashChange': cashChange
        }
    
    accountMovements = pd.DataFrame(map(transactionRecordToAccountMovement, transactions))

    tickersBalance = {}
    
    for ticker, tickerMovements in accountMovements.groupby('ticker'):
        
        tickerMovements = tickerMovements.sort_values('date')
        tickerMovements['cashes'] = np.cumsum(tickerMovements['cashChange'])
        tickerMovements['shares'] = np.cumsum(tickerMovements['shareChange'])
        
        tickersBalance[ticker] = \
            tickerMovements[['date', 'shares', 'cashes']] \
                .drop_duplicates('date', keep='last')

    return tickersBalance


def mergeBalanceWithClosePrice(tickerBalance, ticker):
    
    startDate = min(tickerBalance['date'])
    closePrice = ydf[['Close']]
    #closePrice = yf.Ticker(ticker).history(start='2020-07-02')[['Close']]
    
    return closePrice \
        .merge(tickerBalance, how='outer', left_index = True, right_on='date') \
        .fillna(method='pad') \
        .fillna(0) \
        .reset_index(drop=True)
    
    
    
def calculateTickerDailyStat(tickerBalanceAndPriceDF):
    
    resultDF = tickerBalanceAndPriceDF.reset_index(drop=True)
    
    resultDF['marketValue'] = resultDF['Close'] * resultDF['shares']
    
    resultDF['netProfit'] = resultDF['marketValue'] + resultDF['cashes']
    
    resultDF['dailyProfit'] = np.diff(resultDF['netProfit'], prepend=0)
    
    resultDF['cashFlow'] = np.diff(resultDF['cashes'], prepend=resultDF.head(1)['marketValue'])
    
    startDate = min(resultDF['date'])
    endDate = max(resultDF['date'])
    

    def adjustedCost(df):
        def calculateRowAdjustedCost(row):
            subDF = df.iloc[:row.name+1]
            weight = (row.date - subDF['date'] + timedelta(days=1)) / (row.date - startDate + timedelta(days=1))
            return -(subDF['cashFlow'] * weight).sum()

        return df.apply(calculateRowAdjustedCost, axis=1)
    
    resultDF['adjustedCost'] = adjustedCost(resultDF[['date', 'cashFlow']])
    resultDF['dollarWeightedReturnRate'] = resultDF['netProfit']/ adjustedCost(resultDF[['date', 'cashFlow']])
    
    return resultDF.fillna(0)

In [20]:
tickerBalanceAndPriceDF = mergeBalanceWithClosePrice(transactionRecordsToTickersBalance(transactions)['SOXX'], 'SOXX')

In [21]:
calculateTickerDailyStat(tickerBalanceAndPriceDF)

,Close,date,shares,cashes,marketValue,netProfit,dailyProfit,cashFlow,adjustedCost,dollarWeightedReturnRate
0,265.078217,2020-07-01,0.0,0.0,0.000000,0.000000,0.000000,0.0,-0.000000,0.000000
1,268.319855,2020-07-02,2.0,-600.0,536.639709,-63.360291,-63.360291,-600.0,300.000000,-0.211201
2,275.407806,2020-07-06,2.0,-600.0,550.815613,-49.184387,14.175903,0.0,500.000000,-0.098369
3,272.156219,2020-07-07,2.0,-600.0,544.312439,-55.687561,-6.503174,0.0,514.285714,-0.108281
4,275.843964,2020-07-08,2.0,-600.0,551.687927,-48.312073,7.375488,0.0,525.000000,-0.092023
...,...,...,...,...,...,...,...,...,...,...
254,449.670013,2021-07-06,3.0,-600.0,1349.010040,749.010040,-2.309967,0.0,598.382749,1.251724
255,442.859985,2021-07-07,3.0,-600.0,1328.579956,728.579956,-20.430084,0.0,598.387097,1.217573
256,437.170013,2021-07-08,3.0,-600.0,1311.510040,711.510040,-17.069916,0.0,598.391421,1.189038
257,444.299988,2021-07-09,3.0,-600.0,1332.899963,732.899963,21.389923,0.0,598.395722,1.224775
